### An example MD simulation with a box of LJ particles.  
**Authors: Zilin Song**

#### 0. Imports.

In [ ]:
import numpy as np

import mdpy.box
import mdpy.topology
import mdpy.potentials
import mdpy.system
import mdpy.integrators

#### 1. Basic elements of an MD simulation

In [ ]:
# elements of an MD system.
box         = mdpy.box.PBCBox(xdim=50., ydim=50., zdim=50.)
topology    = mdpy.topology.HomogeneousIdealGas(num_particles=200, mass=1.)
coordinates = box.wrap(coordinates=np.random.randn(200, 3)*100.)

# The MD system.
system = mdpy.system.System(box=box, topology=topology, coordinates=coordinates)
system.add_potential(mdpy.potentials.LJ126(sigma=1., epsilon=1.))

# The MD integrator.
integrator = mdpy.integrators.LangevinIntegrator(system=system, timestep=.05, friction=5., temperature=300.)

#### 2. The MD simulation with temperature monitoring.

In [ ]:
# initialize velocities at 300 K
integrator.initialize_velocities(temperature=300.)
trajectory = []

# integrate for 200 steps.
for i in range(500):
  integrator.step()

  # record the trajectory every 5 steps.
  if i % 10 == 0:
    print(f"Current step: {i+1}, temperature {integrator.compute_temperature():8.4f} K.")
    trajectory.append(box.wrap(integrator.system.coordinates))

In [ ]:
import mdtraj
import nglview

traj = mdtraj.Trajectory(xyz=np.asarray(trajectory), topology=system.topology.as_mdtraj())
view = nglview.show_mdtraj(traj)
view.add_ball_and_stick('all', radius=20.)
view